### 임포트

In [1]:
import torch
import os
import sys
import numpy as np
import random

# custom
from util import *
from transformers import GPT2Tokenizer
from AAC_Prefix.AAC_Prefix import * # network
from Train import *

### 기타 값들 설정

In [2]:
# prefix vector 크기 설정
temporal_prefix_size = 15 # 0 or 15
global_prefix_size = 11 # 0 or 11

prefix_size = temporal_prefix_size + global_prefix_size 

# mapping network가 사용할 transformer의 스펙 설정
transformer_num_layers = {"temporal_num_layers" : 4, "global_num_layers" : 4}
prefix_size_dict = {"temporal_prefix_size" : temporal_prefix_size, "global_prefix_size" : global_prefix_size}


data_dir = './AudioCaps'
MODEL_NAME = 'add_exp_train_audiocaps_test_audiocaps'

epochs = 50
LR = 5e-5

TEST_BATCH_SIZE = 5
TRAIN_BATCH_SIZE = 75

random_seed=2766
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
torch.backends.cudnn.benchmark=False
torch.backends.cudnn.deterministic=True
np.random.seed(random_seed)
random.seed(random_seed)  

USE_CUDA = torch.cuda.is_available() 
device = torch.device('cuda' if USE_CUDA else 'cpu')

### Tokenizer, Dataloader 불러오기

In [3]:
tokenizer_type = 'Custom'
tokenizer = tokenizer_forCustomVocab(Dataset = 'AudioCaps')
vocab_size = len(tokenizer.vocab)

In [4]:
test_dataloader  = CreateDataloader(tokenizer, data_dir, TEST_BATCH_SIZE, 'test', prefix_size, is_TrainDataset = False, tokenizer_type = tokenizer_type)
train_dataloader = CreateDataloader(tokenizer, data_dir, TRAIN_BATCH_SIZE, 'train', prefix_size, is_TrainDataset = True, tokenizer_type = tokenizer_type)

get dataset...: 100%|████████████████████| 49276/49276 [03:00<00:00, 272.83it/s]


### 학습결과 정리하는 폴더 생성하기

In [5]:
directory = "./Train_record/params_" + MODEL_NAME
try:
    if not os.path.exists(directory):
        os.makedirs(directory)
except OSError:
    print("Error: Failed to create the directory.")


### 모델 초기화

In [6]:
model = get_AAC_Prefix(tokenizer, 
                        vocab_size = vocab_size, Dataset = 'AudioCaps',
                        prefix_size_dict = prefix_size_dict, transformer_num_layers = transformer_num_layers, 
                        encoder_freeze = True, decoder_freeze = True,
                        pretrain_fromAudioCaps = False, device = device)

/home/cuai5th/anaconda3/envs/minkyu_env/lib/python3.8/site-packages/torchlibrosa/stft.py:193: FutureWarning: Pass size=512 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = librosa.util.pad_center(fft_window, n_fft)
/home/cuai5th/anaconda3/envs/minkyu_env/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)


use Custom Tokenizer
temporal feature's mapping network : num_head = 8 num_layers = 4
global feature ver's mapping network : num_head = 8 num_layers = 4
use custom header!
Encoder freezing
GPT2 freezing


### 학습 & 평가

In [7]:
Train(model, LR, train_dataloader, test_dataloader,
    epochs, model_name = MODEL_NAME, beam_search = True, device = device,
    Dataset = 'AudioCaps', test_dataloader_other_dataset = None)

/home/cuai5th/anaconda3/envs/minkyu_env/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Eval using dataset...: 100%|██████████████████| 957/957 [03:46<00:00,  4.23it/s]


loading annotations into memory...
0:00:00.006509
creating index...
index created!
Loading and preparing results...     
DONE (t=0.00s)
creating index...
index created!
tokenization...


PTBTokenizer tokenized 59528 tokens at 423690.93 tokens per second.
PTBTokenizer tokenized 8691 tokens at 104160.26 tokens per second.


setting up scorers...
computing Bleu score...
{'testlen': 6778, 'reflen': 7215, 'guess': [6778, 5821, 4868, 3999], 'correct': [3863, 1710, 636, 117]}
ratio: 0.9394317394316092
Bleu_1: 0.534
Bleu_2: 0.384
Bleu_3: 0.262
Bleu_4: 0.149
computing METEOR score...
METEOR: 0.170
computing Rouge score...
ROUGE_L: 0.428
computing CIDEr score...
CIDEr: 0.425
computing SPICE score...


Parsing reference captions
Parsing test captions
Initiating Stanford parsing pipeline
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.TokenizerAnnotator - TokenizerAnnotator: No tokenizer type provided. Defaulting to PTBTokenizer.
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator parse
[main] INFO edu.stanford.nlp.parser.common.ParserGrammar - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz ... 
done [0.4 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [1.4 sec].
Loading classifier from edu/stanford/nlp/models/ner/english.muc.7class.distsim.crf.ser.gz ... done [0.6

...

loading annotations into memory...
0:00:00.006377
creating index...
index created!
Loading and preparing results...     
DONE (t=0.00s)
creating index...
index created!
tokenization...


PTBTokenizer tokenized 59528 tokens at 431194.68 tokens per second.
PTBTokenizer tokenized 8892 tokens at 94572.46 tokens per second.


setting up scorers...
computing Bleu score...
{'testlen': 6954, 'reflen': 7116, 'guess': [6954, 5997, 5040, 4090], 'correct': [4643, 2359, 1118, 382]}
ratio: 0.9772344013489351
Bleu_1: 0.652
Bleu_2: 0.501
Bleu_3: 0.379
Bleu_4: 0.265
computing METEOR score...
METEOR: 0.214
computing Rouge score...
ROUGE_L: 0.484
computing CIDEr score...
CIDEr: 0.601
computing SPICE score...


Parsing reference captions
Parsing test captions
Initiating Stanford parsing pipeline
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.TokenizerAnnotator - TokenizerAnnotator: No tokenizer type provided. Defaulting to PTBTokenizer.
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator parse
[main] INFO edu.stanford.nlp.parser.common.ParserGrammar - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz ... 
done [0.4 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [1.4 sec].
Loading classifier from edu/stanford/nlp/models/ner/english.muc.7class.distsim.crf.ser.gz ... done [0.6

SPICE evaluation took: 9.894 s
SPICE: 0.162
computing SPIDEr score...
SPIDEr: 0.381

Training time : 4:13:18
